In [25]:
import openai
import instructor
from pydantic import BaseModel, Field
from qdrant_client import QdrantClient


In [13]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: what is the capital of France?
"""

In [14]:
response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": prompt}],
)


In [15]:
print(response.choices[0].message.content)

The capital of France is Paris.


In [16]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="The answer to the question")


In [31]:
client = instructor.from_openai(openai.OpenAI())

response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": prompt}],
    response_model=RAGGenerationResponse,
)


In [20]:
response.model_dump_json()


'{"answer":"The capital of France is Paris."}'

In [21]:
raw_response


ChatCompletion(id='chatcmpl-CXCBgzF5eV6mpD6SJgPX1JN0JkVtE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_ZkFd8d6eYhdUUg2OIlbrmmzJ', function=Function(arguments='{"answer":"The capital of France is Paris."}', name='RAGGenerationResponse'), type='function')]))], created=1762027708, model='gpt-4.1-mini-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_4c2851f862', usage=CompletionUsage(completion_tokens=11, prompt_tokens=97, total_tokens=108, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=None), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [26]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [35]:
def get_embeddings(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        model=model,
        input=text
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):
    """
    Retrieve k most similar items to the query from Qdrant collection.
    """
    response = qdrant_client.query_points(
        collection_name="amazon_items-collection-00",
        query=get_embeddings(query),
        limit=k
    )

    retrieved_context_ids = [];
    # this is description of the product
    retrieved_context = [];
    similarity_scores = [];
    retrieved_content_ratings = []
    for point in response.points:   
        retrieved_context_ids.append(point.payload["parent_asin"])
        retrieved_context.append(point.payload["description"])
        similarity_scores.append(point.score)
        retrieved_content_ratings.append(point.payload["average_rating"])
    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
        "retrieved_content_ratings": retrieved_content_ratings
    }

def format_retrieved_context(context_data):
    formatted_context = ""
    for id, context, rating in zip(context_data["retrieved_context_ids"], context_data["retrieved_context"], context_data["retrieved_content_ratings"]):
            formatted_context += f"- {id}, description: {context}, rating: {rating}\n"  
    return formatted_context

def create_prompt(query, preprocessed_retrieved_context):
    processed_context = format_retrieved_context(preprocessed_retrieved_context)
    
    prompt = f"""
    You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".

    Question: 
    {query}
    Context: 
    {processed_context}
    """
    return prompt

def generate_answer(prompt):
    client = instructor.from_openai(openai.OpenAI())
    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
        response_model=RAGGenerationResponse
    )

    return response

def rag_pipeline(query, qdrant_client, top_k=5):
    preprocessed_retrieved_context = retrieve_data(query, qdrant_client, top_k)
    prompt = create_prompt(query, preprocessed_retrieved_context)
    answer = generate_answer(prompt)

    final_response = {
        "question": query,
        "answer": answer.answer,
        "retrieved_context_ids": preprocessed_retrieved_context["retrieved_context_ids"],
        "retrieved_context": preprocessed_retrieved_context["retrieved_context"],
        "similarity_scores": preprocessed_retrieved_context["similarity_scores"]
    }

    return final_response

In [36]:
output = rag_pipeline("What is the most popular product?", qdrant_client)

In [37]:
output

{'question': 'What is the most popular product?',
 'answer': 'The most popular product based on the ratings provided is the 1TB USB Flash Drive, NEIGHBOR Portable Thumb Drive with a rating of 4.8.',
 'retrieved_context_ids': ['B0CH6P8DYF',
  'B0BFPZGYLD',
  'B0CF57H28T',
  'B09LVX3XW2',
  'B0BP9Z159S'],
 'retrieved_context': ['New 2023 80x100 High Powered Monoculars for Adults high Powered BAK-4 Prism and FMC Lens Monocular Telescope for Smartphone Monoculars for Bird Watching/Wildlife/Hunting/Hiking 【80x100 High Power Monocular】100 mm object diameter monocular, 80 times magnification. Using optical technology, this monocular uses a fully coated lens to ensure excellent light transmittance and image brightness, provide images with high definition, high quality and high color saturation. 【Compact Monocular】Compact and light monocular brings you the distant scenery by adjusting the eye cup and focusing wheel. The monocular is small and easy to carry, and can also obtain stable and clear 